<a href="https://colab.research.google.com/github/LhsEric/dscareer_dsml_exercies_coach/blob/main/Task_03_%E4%BD%9C%E6%A5%AD_02%E3%80%90%E5%AF%A6%E4%BD%9C%E9%A1%8C%E3%80%91%E5%B0%87%E8%B3%87%E6%96%99%E5%AD%98%E5%85%A5%E8%B3%87%E6%96%99%E5%BA%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Task #03 作業 02【實作題】將資料存入資料庫

練習：
在前述的練習中，你已經利用爬蟲成功的收集到了來自台灣銀行牌告匯率的資料，但我們現在每天只會抓取到一筆當日的資料。
如果我們希望能夠觀察趨勢的話，會希望能夠週期性的抓取並且儲存下來。接下來這個任務是請你將這個結果紀錄再加上更新時間的欄位，將資料存到到 SQLite 資料庫（名稱欄位自訂）中。


* Sample Input：https://rate.bot.com.tw/xrt?Lang=zh-TW（資料來源：台灣銀行牌告匯
* 將匯率資訊加上更新更新時間欄位並且將資料存到到 SQLite 資料庫中

In [15]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3


url = 'https://rate.bot.com.tw/xrt?Lang=zh-TW'

r = requests.get(url)
response = r.text
d = BeautifulSoup(response, 'html.parser')
# print(d)

# 物件來取出
currency = [currency.text.strip() for currency in d.find_all("div", class_="visible-phone print_hide")]
cash_exchange_sell = [cash_exchange_sell.text.strip() for cash_exchange_sell in d.find_all("td", {"data-table": "本行現金買入",
                                                "class": "rate-content-cash text-right print_hide"})]
cash_exchange_buy = [cash_exchange_buy.text.strip() for cash_exchange_buy in d.find_all("td", {"data-table": "本行現金賣出",
                                                "class": "rate-content-cash text-right print_hide"})]
fx_spot_sell = [fx_spot_sell.text.strip() for fx_spot_sell in d.find_all("td", {"data-table": "本行即期買入",
                                                "class": "rate-content-sight text-right print_hide"})]
fx_spot_buy = [fx_spot_buy.text.strip() for fx_spot_buy in d.find_all("td", {"data-table": "本行即期賣出",
                                                "class": "rate-content-sight text-right print_hide"})]

# 把數據整理成 DataFrame
data = {
    '幣別': currency,
    '現金匯率本行賣出': cash_exchange_sell,
    '現金匯率本行買入': cash_exchange_buy,
    '即期匯率本行賣出': fx_spot_sell,
    '即期匯率本行買入': fx_spot_buy
    }

df = pd.DataFrame(data)
# print(df)

# 建立 SQLite 資料庫連線
conn = sqlite3.connect('currency.db')

# 建立一個名為 currency 的資料表
df.to_sql('currency_rate', conn, if_exists='append', index=False)


# 關閉資料庫連接
conn.close()



            幣別 現金匯率本行賣出 現金匯率本行買入 即期匯率本行賣出 即期匯率本行買入
0     美金 (USD)       32    32.67   32.325   32.475
1     港幣 (HKD)    3.989    4.193     4.11     4.18
2     英鎊 (GBP)    40.02    42.14   40.915   41.545
3     澳幣 (AUD)    21.12     21.9   21.335    21.68
4   加拿大幣 (CAD)    23.23    24.14    23.56    23.89
5   新加坡幣 (SGD)    23.38    24.29    23.85    24.07
6   瑞士法郎 (CHF)    34.99    36.19     35.6    35.99
7     日圓 (JPY)   0.1969   0.2097   0.2037   0.2087
8    南非幣 (ZAR)        -        -    1.675    1.765
9    瑞典幣 (SEK)     2.68      3.2     3.01     3.13
10    紐元 (NZD)    19.36    20.21    19.69    19.99
11    泰幣 (THB)   0.7527   0.9427   0.8621   0.9081
12  菲國比索 (PHP)   0.4878   0.6198        -        -
13   印尼幣 (IDR)  0.00158  0.00228        -        -
14    歐元 (EUR)    34.28    35.62   34.795   35.395
15    韓元 (KRW)  0.02169  0.02559        -        -
16   越南盾 (VND)  0.00103  0.00144        -        -
17   馬來幣 (MYR)    5.843    7.363        -        -
18   人民幣 (CNY)    4.365    4.52